In [1]:
import pandas as pd
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
import os
import subprocess

In [2]:
galaxyzoo = pd.read_csv("C:\\Users\\awe2\DL_DES-master\\data\\sdss-galaxyzoo\\high_certainty\\sdss_metadata\\metadata-bat0008.csv")
#GZ2 classified gri colour composite images selected on the basis of magnitude (mr < 17), angular size (r90 > 3 arcsec), 
#and redshift (0.0005 <z< 0.25) criteria. Deeper images from Stripe 82

In [3]:
def SDSS_objid_to_values(objid):
    bin_objid = bin(objid)
    bin_objid = bin_objid[2:len(bin_objid)]
    bin_objid = bin_objid.zfill(64)

    empty = int( '0b' + bin_objid[0], base=0)
    skyVersion = int( '0b' + bin_objid[1:4+1], base=0)
    rerun = int( '0b' + bin_objid[5:15+1], base=0)
    run = int( '0b' + bin_objid[16:31+1], base=0)
    camcol = int( '0b' + bin_objid[32:34+1], base=0)
    firstField = int( '0b' + bin_objid[35+1], base=0)
    field = int( '0b' + bin_objid[36:47+1], base=0)
    object_num = int( '0b' + bin_objid[48:63+1], base=0)

    #return skyVersion, rerun, run, camcol, field, object_num
    return (str(run), str(camcol), str(field), str(rerun))

In [4]:
objid = galaxyzoo['objid'].values
ra = galaxyzoo['ra'].values
dec = galaxyzoo['dec'].values

In [5]:
out_dir = "New_Kahn_fits//raw//"
for i in range(len(objid)):
    #run, camcol, field, rerun = SDSS_objid_to_values(objid[i])
    for filter_ in ['u','g','r','i','z'] :
        run, camcol, field, rerun = SDSS_objid_to_values(objid[i])
        req_url = ('http://das.sdss.org/cgi-bin/drC?RUN={}&RERUN={}&CAMCOL={}&FIELD={}&FILTER={}'.format(run,
                                                                                                         rerun,
                                                                                                         camcol,
                                                                                                         field,
                                                                                                         filter_))
        out_path = os.path.join(out_dir,
                'run{}-rerun{}-camrol{}-field{}-{}.fits'.format(run, rerun, camcol, field, filter_))
        subprocess.run(['wget', req_url, '-O', out_path, '-q'])

In [6]:
#two examples of 120x120 jpeg cutouts centered on galaxies...

#http://skyserver.sdss.org/dr13/en/tools/chart/navi.aspx?ra=136.9616&dec=16.6383&scale=0.1&width=120&height=120&opt=

#http://skyserver.sdss.org/dr13/en/tools/chart/navi.aspx?ra=136.9616&dec=16.6383&scale=0.2&width=120&height=120&opt=

#generates jpegs of what we want, scale is (arcsec/px) /2. If we can find a table of petrosian radii we can use that to set 
#appropriate value, generate URL download links, and then hit it. essentially, according to SDSS website, 2 petroRAD is enough
#to capture all flux from a galaxy and reduce background noise; we might use 3-5 * petroRad and then bin into appropriate px
#sclaes off that. http://cas.sdss.org/dr7/en/help/docs/algorithm.asp?search=petrosian+magnitude&submit1=Search